In [1]:
model_name = "davnas/Italian_Cousine_1.3"
dataset_name = "/content/test_set_new.jsonl"


In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

!pip install -q bitsandbytes
!pip install -q evaluate jsonlines rouge_score bert-score
!pip install transformers peft accelerate bitsandbytes jsonlines
!pip install evaluate tqdm
import evaluate
import evaluate
import jsonlines
from unsloth import FastLanguageModel
import torch
from tqdm import tqdm  # Import tqdm for progress tracking


In [3]:
# prompt: please download the test set from "https://github.com/davidenascivera/LAB2_Scalable/blob/main/Test_set/test_set.jsonl"

import requests
from io import StringIO
import json

url = "https://raw.githubusercontent.com/davidenascivera/LAB2_Scalable/main/Test_set/test_set.jsonl"

try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open("test_set.jsonl", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download successful. File saved as test_set.jsonl")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

Download successful. File saved as test_set.jsonl


In [4]:
# prompt: please download the test set from "https://github.com/davidenascivera/LAB2_Scalable/blob/main/Test_set/test_set.jsonl"

import requests
from io import StringIO
import json

url = "https://raw.githubusercontent.com/davidenascivera/LAB2_Scalable/main/Test_set/test_set_new.jsonl"

try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open("test_set_new.jsonl", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download successful. File saved as test_set.jsonl")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

Download successful. File saved as test_set.jsonl


## 1.SimpleInference

In [5]:
"""
from transformers import AutoModel, AutoTokenizer

max_seq_length = 2048  # Choose any! We auto support ROPE scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, bFloat16 for Ampere+

model_name_or_path = "davnas/Italian_Cousine"

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name_or_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
    # token = "hf_...", #se il nostro modello non è public
    # Use one if using gated models like meta-llama/Llama-2-7b-hf
)
"""

'\nfrom transformers import AutoModel, AutoTokenizer\n\nmax_seq_length = 2048  # Choose any! We auto support ROPE scaling internally!\ndtype = None  # None for auto detection. Float16 for Tesla T4, V100, bFloat16 for Ampere+\n\nmodel_name_or_path = "davnas/Italian_Cousine"\n\nfrom unsloth import FastLanguageModel\nimport torch\n\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    model_name=model_name_or_path,\n    max_seq_length=max_seq_length,\n    dtype=dtype,\n    load_in_4bit=True,\n    # token = "hf_...", #se il nostro modello non è public\n    # Use one if using gated models like meta-llama/Llama-2-7b-hf\n)\n'

In [6]:
"""
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How can I cook an smoothie?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128,
               use_cache=True, temperature=1.5, min_p=0.1)
"""

'\nfrom unsloth import FastLanguageModel\n\nFastLanguageModel.for_inference(model)  # Enable native 2x faster inference\n\nmessages = [\n    {"role": "user", "content": "How can I cook an smoothie?"},\n]\n\ninputs = tokenizer.apply_chat_template(\n    messages,\n    tokenize=True,\n    add_generation_prompt=True,  # Must add for generation\n    return_tensors="pt",\n).to("cuda")\n\nfrom transformers import TextStreamer\ntext_streamer = TextStreamer(tokenizer, skip_prompt=True)\n\nmodel.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128,\n               use_cache=True, temperature=1.5, min_p=0.1)\n'

## 2.Evaluation

In [7]:
def load_test_data(file_path):
    """Load test data from JSONL file."""
    prompts = []
    completions = []
    with jsonlines.open(file_path) as reader:
        for item in reader:
            prompts.append(item['prompt'])
            completions.append(item['completion'])
    return prompts, completions



In [8]:
# prompt: clean the memory vram

import gc
import torch

# Clear GPU memory
torch.cuda.empty_cache()

# Run garbage collection
gc.collect()


60

In [9]:

#------------------------------------------

def generate_predictions(model, tokenizer, prompts):
    """Generate model predictions for given prompts with progress tracking."""
    predictions = []

    # Use tqdm to wrap the prompts list for progress tracking
    for i, prompt in enumerate(tqdm(prompts, desc="Generating Predictions")):
        print(f"Processing prompt {i + 1}/{len(prompts)}: {prompt}")

        messages = [{"role": "user", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        # Ensure input_ids are Long type and on cuda
        inputs = inputs.to("cuda").to(torch.long)  # Changed this line

        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=512,
            use_cache=True,
            temperature=1.5,
            min_p=0.1
        )

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)

    return predictions

# Initialize model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,  # Use the actual model name
    max_seq_length=2048,
    load_in_4bit=True
)
FastLanguageModel.for_inference(model)

# Load test data
prompts, references = load_test_data(dataset_name)

# Generate predictions with progress tracking
predictions = generate_predictions(model, tokenizer, prompts)


==((====))==  Unsloth 2024.12.3: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Generating Predictions:   0%|          | 0/40 [00:00<?, ?it/s]

Processing prompt 1/40: How can I make a hearty vegetable stir-fry?


Generating Predictions:   2%|▎         | 1/40 [00:13<08:42, 13.40s/it]

...

Generating Predictions:  95%|█████████▌| 38/40 [04:20<00:12,  6.07s/it]

Processing prompt 39/40: How do I make a simple bruschetta appetizer?


Generating Predictions:  98%|█████████▊| 39/40 [04:26<00:06,  6.00s/it]

Processing prompt 40/40: What’s the best way to make a shrimp and avocado salad?


Generating Predictions: 100%|██████████| 40/40 [04:31<00:00,  6.79s/it]


In [10]:
# Evaluate using ROUGE
rouge = evaluate.load('rouge')
rouge_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)

# Evaluate using BERTScore
bertscore = evaluate.load('bertscore')
bert_results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli"
)

print(f"Dataset: {dataset_name}")
print(f"Model: {model_name}")

# Print results
print("\nROUGE Scores:")
for metric, score in rouge_results.items():
    print(f"{metric}: {score:.4f}")

print("\nBERTScore:")
print(f"Precision: {sum(bert_results['precision']) / len(bert_results['precision']):.4f}")
print(f"Recall: {sum(bert_results['recall']) / len(bert_results['recall']):.4f}")
print(f"F1: {sum(bert_results['f1']) / len(bert_results['f1']):.4f}")

Dataset: /content/test_set_new.jsonl
Model: davnas/Italian_Cousine_1.3

ROUGE Scores:
rouge1: 0.4347
rouge2: 0.1312
rougeL: 0.2832
rougeLsum: 0.4289

BERTScore:
Precision: 0.6936
Recall: 0.6919
F1: 0.6920
